In [14]:
import pygame
pygame.init()
window_size = (pygame.display.Info().current_w -100, pygame.display.Info().current_h - 100)
screen = pygame.display.set_mode(window_size)
pygame.display.set_caption("Egg World")
done = False
clock = pygame.time.Clock()
BLUE = (0,0,255)
x = 0 
y = 0
while not done:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done = True
    screen.fill((0,0,0))
    pygame.draw.rect(screen, BLUE, (x, y, 200, 500))
    pygame.display.flip()
    if y or x < dimensions_y-100:
        y+=1
        x+=1
    clock.tick(50)
pygame.quit()



In [16]:
def xy_position_to_pixels(Rectangle, window_size):
    """Takes:In-game coordinate rectangle
    Converts in game coordinates to pixels. Also, note that the in-game position refers to the LOWER RIGHT hand
    corner of the sprite. 
    Returns: rectangle in pixel coordinates"""
    x = Rectangle[0]
    y = Rectangle[1]
    width = Rectangle[2]
    height = Rectangle[3]
    pixel_x = float(x/300)*window_size[0]
    pixel_y = float(y/200)*window_size[1]
    pixel_width = float(width/300)*window_size[0]
    pixel_height = float(height/200)*window_size[1]
    pixel_x -= pixel_width
    return (pixel_x, pixel_y, pixel_width, pixel_height)

def give_next_position(current_x, current_y, current_xs, current_ys, floor, button_states):
    if current_y
        
        
    

SyntaxError: invalid syntax (<ipython-input-16-cf3afaa8533b>, line 14)